========================================================================================
PROBLEM B3
Build a CNN based classifier for Rock-Paper-Scissors dataset.
Your input layer should accept 150x150 with 3 bytes color as the input shape.
This is unlabeled data, use ImageDataGenerator to automatically label it.
Don't use lambda layers in your model.
The dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
Desired accuracy AND validation_accuracy > 83% 
========================================================================================

In [1]:
import urllib.request
import zipfile
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_url = 'https://github.com/dicodingacademy/assets/releases/download/release-rps/rps.zip'
urllib.request.urlretrieve(data_url, 'rps.zip')
local_file = 'rps.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [2]:
TRAINING_DIR = "data/rps"
train_datagen = ImageDataGenerator(
    rescale= 1./255.,
    rotation_range= 40,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    fill_mode= 'nearest',
    validation_split=0.3
)
# YOUR IMAGE SIZE SHOULD BE 150x150
# Make sure you used "categorical"
# YOUR CODE HERE
train_generator= train_datagen.flow_from_directory(TRAINING_DIR,
                                                   batch_size=16,
                                                   subset='training',
                                                   color_mode='rgb',
                                                   class_mode='categorical',
                                                   target_size=(150, 150))

validation_datagen = ImageDataGenerator(
    rescale= 1. /255.,
    validation_split=0.3
)
validation_generator = validation_datagen.flow_from_directory(TRAINING_DIR,
                                                              batch_size=16,
                                                              subset='validation',
                                                              color_mode='rgb',
                                                              class_mode='categorical',
                                                              target_size=(150, 150))

Found 1764 images belonging to 3 classes.
Found 756 images belonging to 3 classes.


In [12]:
class Custom_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('val_accuracy') > 0.91):
            print("\nVal Accuracy > 91%")
            print("\nTraining Selesai")
            self.model.stop_training = True

In [13]:
# YOUR CODE HERE, end with 3 Neuron Dense, activated by softmax
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [14]:
callbacks = Custom_callback()

validation_steps = validation_generator.samples / validation_generator.batch_size - 1

model.fit(train_generator, 
          validation_data = validation_generator,
          epochs=20,
          steps_per_epoch= 20,
          validation_steps=validation_steps,
          callbacks=callbacks
          )

Epoch 1/20
20/20 [==============================] - 6s 225ms/step - loss: 1.9036 - accuracy: 0.3281 - val_loss: 1.1049 - val_accuracy: 0.3298
Epoch 2/20
20/20 [==============================] - 5s 231ms/step - loss: 1.3667 - accuracy: 0.3125 - val_loss: 1.0967 - val_accuracy: 0.3338
Epoch 3/20
20/20 [==============================] - 4s 222ms/step - loss: 1.1136 - accuracy: 0.4156 - val_loss: 1.0904 - val_accuracy: 0.3311
Epoch 4/20
20/20 [==============================] - 4s 209ms/step - loss: 1.1067 - accuracy: 0.3625 - val_loss: 1.1106 - val_accuracy: 0.3338
Epoch 5/20
20/20 [==============================] - 4s 209ms/step - loss: 1.1371 - accuracy: 0.4906 - val_loss: 1.2125 - val_accuracy: 0.3418
Epoch 6/20
20/20 [==============================] - 4s 210ms/step - loss: 1.0234 - accuracy: 0.5406 - val_loss: 0.7258 - val_accuracy: 0.5891
Epoch 7/20
20/20 [==============================] - 4s 209ms/step - loss: 1.0803 - accuracy: 0.5750 - val_loss: 0.7530 - val_accuracy: 0.5838
Epoch 

In [15]:
model.save("model_B3.h5")